In [ ]:
!tensorboard --logdir=/opt/ml/insightface/recognition/arcface_torch/work_dirs/wf4m_r50/tensorboard

In [ ]:
from deepface import 

In [10]:
from torch.utils.data import Dataset, DataLoader
from Facial_verification.Crawling_Dataset import Crawling_Nomal_Dataset
import torch
import cv2 
import os
import numpy as np
from torchvision import transforms
import torch.utils.data as data
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
import math
import tqdm as tqdm
import matplotlib.pyplot as plt
from PIL import Image
from collections import defaultdict

In [11]:
test_path = '/opt/ml/data/celeb_30/cut_test'
train_path = '/opt/ml/data/celeb_30/cut_train'

In [13]:
transform = transforms.Compose([
        np.float32,
        transforms.ToTensor(),  # range [0, 255] -> [0.0, 1.0]
        fixed_image_standardization
    ])

In [15]:
test_dataset = Crawling_Nomal_Dataset(test_path, transforms=transform)
test_embedding_loader = data.DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=2, drop_last=False)

identities = defaultdict(lambda : list())

for img,  path_label_list in test_embedding_loader :
    img_path = path_label_list[0][0]
    img_label = path_label_list[1][0]
    identities[img_label].append(img_path)

In [18]:
from insightface.recognition.arcface_torch import IResNet, IBasicBloc

ImportError: cannot import name 'IResNet' from 'insightface.recognition.arcface_torch' (unknown location)

In [ ]:
model_pth = '/opt/ml/insightface/recognition/arcface_torch/work_dirs/wf4m_r50/model.pt'

In [ ]:
model = IResNet(IBasicBlock, [3, 4, 14, 3], fp16=True, num_features=512)
model.load_state_dict(torch.load(model_pth))

In [ ]:
import pandas as pd

In [ ]:
positives = []
for key, values in idendities.items():
 for i in range(0, len(values)-1):
  for j in range(i+1, len(values)):
   positive = []
   positive.append(values[i])
   positive.append(values[j])
   positives.append(positive)
 
positives = pd.DataFrame(positives, columns = ["file_x", "file_y"])
positives["decision"] = "Yes"

In [ ]:
samples_list = list(idendities.values())
 
negatives = []
for i in range(0, len(idendities) - 1):
 for j in range(i+1, len(idendities)):
  cross_product = itertools.product(samples_list[i], samples_list[j])
  cross_product = list(cross_product)
 
  for cross_sample in cross_product:
   negative = []
   negative.append(cross_sample[0])
   negative.append(cross_sample[1])
   negatives.append(negative)
 
negatives = pd.DataFrame(negatives, columns = ["file_x", "file_y"])
negatives["decision"] = "No"

In [20]:
df = pd.concat([positives, negatives]).reset_index(drop = True)
 
df.file_x = "dataset/"+df.file_x
df.file_y = "dataset/"+df.file_y

NameError: name 'pd' is not defined

In [19]:
# get distance
from deepface import DeepFace
 
instances = df[["file_x", "file_y"]].values.tolist()
resp_obj = DeepFace.verify(instances, model_name = "VGG-Face", distance_metric = "cosine")

2023-07-13 00:54:16.602735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 00:54:17.530008: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


NameError: name 'df' is not defined

In [ ]:
distances = []
for i in range(0, len(instances)):
 distance = round(resp_obj["pair_%s" % (i+1)]["distance"], 4)
 distances.append(distance)
 
df["distance"] = distances

In [ ]:
# analysis distance
tp_mean = round(df[df.decision == "Yes"].mean().values[0], 4)
tp_std = round(df[df.decision == "Yes"].std().values[0], 4)
fp_mean = round(df[df.decision == "No"].mean().values[0], 4)
fp_std = round(df[df.decision == "No"].std().values[0], 4)

In [ ]:
df[df.decision == "Yes"].distance.plot.kde()
df[df.decision == "No"].distance.plot.kde()

In [ ]:
# decision trees
from chefboost import Chefboost as chef
config = {'algorithm': 'C4.5'}
tmp_df = df[['distance', 'decision']].rename(columns = {"decision": "Decision"}).copy()
model = chef.fit(tmp_df, config)

In [ ]:
def findDecision(distance):
   if distance<=0.3147:
      return 'Yes'
   elif distance>0.3147:
      return 'No'

In [ ]:
# verification
df["prediction"] = "No" #init
idx = df[df.distance <= threshold].index
df.loc[idx, 'prediction'] = 'Yes'

In [ ]:
# evaluation
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df.decision.values, df.prediction.values)
print(cm)
tn, fp, fn, tp = cm.ravel()
recall = tp / (tp + fn)
precision = tp / (tp + fp)
accuracy = (tp + tn)/(tn + fp +  fn + tp)
f1 = 2 * (precision * recall) / (precision + recall)